In [6]:
import nltk
nltk.download('punkt')

from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /home/emmanuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
class Node:
    def __init__(self, item):
        self.item = item
        self.next = None

class LL:
    def __init__(self):
        self.head = None
    def add_node(self,val):
        newNode = Node(val)
        if not self.head:
            self.head = newNode
            return
        temp = self.head
        while temp:
            if val == temp.item:
                return
            temp = temp.next
        if self.head.item > val:
            newNode.next = self.head
            self.head = newNode
            return
        temp = self.head
        while temp and temp.next:
            if val < temp.next.item:
                break
            temp = temp.next
        newNode.next = temp.next
        temp.next = newNode
                    
    def print_list(self):
        tmp = self.head
        while tmp:
            print(tmp.item, end=' ')
            tmp = tmp.next

In [7]:
import pickle

with open("inverted_index.pkl", "rb") as f:
    inverted_index= pickle.load(f)

with open("positional_index.pkl", "rb") as f:
    positional_index = pickle.load(f)

In [9]:
import re
def evalPosQuery(term1, term2, k):
    answer = []
    def posIntersect(p1, p2, k, doc):
        l = []
        pp1 = p1.head
        pp2 = p2.head
        while pp1 is not None:
            while pp2 is not None:
                if abs(pp1.item - pp2.item) <= k:
                    l.append(pp2.item)
                    print(pp1.item, pp2.item)
                elif pp2.item > pp1.item:
                    break
                pp2 = pp2.next
            while l != [] and abs(l[0] - pp1.item) > k:
                l.pop(0)
            for ps in l:
                answer.append([doc, pp1.item, ps])
            pp1 = pp1.next
    if positional_index.get(term1) and positional_index.get(term2):
        for doc in docs:
            print(doc)
            if positional_index[term1].get(doc) and positional_index[term2].get(doc):
                posIntersect(positional_index[ps.stem(term1)][doc], positional_index[ps.stem(term2)][doc], k, doc)
            else:
                continue
    print(answer)    
        

posQuery = input('Enter position query: ')
tkns = posQuery.split(' ')

evalPosQuery(ps.stem(tkns[0]), ps.stem(tkns[1]), int(re.search(r'\d+', tkns[2]).group()))

Enter position query:  Hello World /2


[]
